# Recomendador SFN (sin feedback negativo):
> Este notebook tiene un approach para la realizacion de un Recomendador SFN. 

La idea es que $\hat{Y} \ne X.I^T$, ahora pensamos una prediccion en funcion a la cercania entre puntos de ambas matrices. 

## 1. Transformar la metadata en la matríz $I$ asociada a items:

* METADATA $\rightarrow$ RED NEURONAL $\rightarrow I \in \Re^{MxK}$ 

En este ejemplo vamos a usar a una red neuronal para la transformación, pero podría ser otro método. 

Sea $V_i = (y_0^i, ..., y_k^i)$ ~ "el vector asociado al item i", tenemos:

$I = \begin{pmatrix}
        y_0¹ & ... & y_k^1\\
        \vdots & \vdots & \vdots\\
        y_0^M & ... & y_k^M
    \end{pmatrix} = \begin{pmatrix} V_1 \\ \vdots \\ V_M \end{pmatrix} \in \Re^{MxK}$


## 2. Inicializar la matriz de usuarios $\hat{X}$:

Para cada usuario vamos a armar un vector en el mismo espacio vectorial que $I$ tal que: 

$X_j = (x_0^i, ..., x_k^i)$ ~ "el vector asociado al usuario j"


$\hat{X} = \begin{pmatrix}
        x_0¹ & ... & x_k^1\\
        \vdots & \vdots & \vdots\\
        x_0^M & ... & x_k^M
    \end{pmatrix} = \begin{pmatrix} X_1 \\ \vdots \\ X_M \end{pmatrix} \in \Re^{MxK}$

##### Todos los vectores $X_j$ se inicializan en la media de $I$:

$X_j = (\mu_{x_0}, ..., \mu_{x_k}) = (\mu_{x_0}, ..., \mu_{x_k}) = (\frac{\sum_{l=1}^M I_{l,0}}{M}, ..., \frac{\sum_{l=1}^M I_{l,k}}{M})$ 

## 3. Reubicacion de usuarios:

Por cada item $i$ que vio el usuario $j$, acercamos a $X_j$ a $V_i$:

- $X_j = X_j + \alpha (V_i - X_j) = X_j(1 - \alpha) + \alpha V_i$

Con $\alpha \in [0,1]$ una constante a optimizar.

#### En general vamos a usar:

* Llamemos $I_j(dataset)$ a la lista de los items que vio el usuario $J$ en el dataset.

* Sea $r = $#$I_j(dataset)$ ~ "cantidad de items en $I_j(dataset)$"

* $X_j = X_j + \alpha (\frac{1}{r}\sum_{i\in I_j(dataset)}V_i - X_j)$

O más prolijo, 

$X_j = X_j + \alpha (\mu_{I_j(dataset)} - X_j)$

Con $\mu_{I_j(dataset)}$ ~ "la media de los items vistos"

## 4. Predicciones del modelo $\hat{Y}(\lambda)$:

Como dijimos al principio, la cercanía de los puntos en el espacio vectorial indica similitud.


##### a. Enfoque binario:
- Pensamos a $\hat{Y}$ como una función partida dependiente de la distancia de dos puntos seleccionados, según un parámetro $\lambda$.

- $
  Y^{j,i} = \left \{
              \begin{aligned}
                1 &,\ \text{si} \ |d(X_j, V_i)| <= \lambda\\
                0 &,\ \text{si} \ |d(X_j, V_i)| > \lambda
              \end{aligned}
            \right .
$, con $d(X_j, V_i)$ la distancia euclidiana entre ambos puntos.

- Graficamente, podemos pensar a $\lambda$ como el radio de una esfera para k=3 o de una circunferencia para k=2. A mayor $\lambda \rightarrow$ mayor cantidad de items recomendados.

<image src="sources\lambda_parameter.png" height=300/>

#### b. Scoring:

Podemos hacer un scoring de los items / usuarios B mas cercanos dado un usuario o item A, de la misma forma:

Score(A, B) = $d(A, B) = \sqrt{(A_0-B_0)^2 + ... + (A_k-B_k)^2}$

A menor scoring, mejor la recomendacion de B.


## 5. Testing:

Reubicamos a $\hat{X}$ con trainData. Ahora con el testData vamos a verificar que tan acertado es el modelo.

$Error = \frac{1}{n}\sum_{i,j \in testData}(Y_{i,j}-\hat{Y}_{i,j}(\lambda))$

# Implementacion

### 1. Transformar la metadata en la matriz $I$

In [1]:
import numpy as np 

In [2]:
class Point:
    def __init__(self, id:int, vector:np.array):
        self.id = id
        self.vector = vector
    
    def __dict__(self) -> dict():
        return { self.id : self.vector }
    
class Points:
    def __init__(self):
        self.dict = dict()
        self.ids = list()

    def addPoint(self, point:Point):
        self.ids.append(point.id)
        self.dict.update(point.__dict__())

    def getPointMatrix(self):
        return np.array([self.dict[key]["vector"] for key in self.ids])

    def meanPoint(self) -> np.array:
        pointMatrix = self.getPointMatrix()
        return pointMatrix.mean(axis=0)
    
    def addPoints(self, points_dict:dict):
        for key in list(points_dict.keys()):
            self.dict[key] = points_dict[key]

    def numPoints(self) -> int:
        return len(list(self.dict.keys()))

In [3]:
class Item(Point):
    def __init__(self, group_id, vector):
        super().__init__(group_id, vector)
        self.group_id = group_id

    def __dict__(self) -> dict():
        return { self.group_id : {
                    "vector" : self.vector,
                    "type": "item"
                    }
                }

class User(Point):
    def __init__(self, id, vector):
        super().__init__(id, vector)
        self.user_id = id

    def __dict__(self) -> dict():
        return { self.user_id : {
                    "vector" : self.vector,
                    "type": "user"
                    }
                }

In [4]:
class UserData(Points):
    def addUser(self, user:User):
        self.ids.append(user.user_id)
        self.dict.update(user.__dict__())

class ItemData(Points):
    def addItem(self, item:Item):
        self.ids.append(item.group_id)
        self.dict.update(item.__dict__())    

In [5]:
### Aca necesito leer la metadata de algun lado y transformarla en los vectores

"""
for item_metadata in metadata_index:
    group_id, useful_metadata = process(metadata)
    item_vector = metadata_model.predict(useful_metadata)
    item = Item(group_id, item_vector)
    itemData.addItem(item)
"""

itemA = Item(101010, np.array([1,2,3]))
itemB = Item(202020, np.array([1,2,4]))

itemData = ItemData()
itemData.addItem(itemA)
itemData.addItem(itemB)

itemData.dict

{101010: {'vector': array([1, 2, 3]), 'type': 'item'},
 202020: {'vector': array([1, 2, 4]), 'type': 'item'}}

### 2. Inicializar la matriz de usuarios $\hat{X}$:

In [6]:
mean_item = itemData.meanPoint()
userData = UserData()

# tanto train como test (dataset completo)
users_in_dataset = [1, 2, 3]

for user_id in users_in_dataset:
    user = User(user_id, mean_item)
    userData.addUser(user)

userData.dict

{1: {'vector': array([1. , 2. , 3.5]), 'type': 'user'},
 2: {'vector': array([1. , 2. , 3.5]), 'type': 'user'},
 3: {'vector': array([1. , 2. , 3.5]), 'type': 'user'}}

### 3. Reubicacion de usuarios:

In [7]:
# user, group
train_data = [(1,101010),(2,202020)]
test_data = [(1,202020),(1,101010)]

In [8]:
alpha = 0.6

# Va a ser diferente con lectura de Cassandra
def decode_view(view:tuple) -> tuple:
    return view

for j,view in enumerate(train_data):
    user_id, group_id = decode_view(view)
    userData.dict[user_id]["vector"] += alpha * (itemData.dict[group_id]["vector"] - userData.dict[user_id]["vector"])
    pass

### Armo la matriz con todos los puntos juntos

In [9]:
A = userData.getPointMatrix()
B = itemData.getPointMatrix()

UserItemPoints = Points()
UserItemPoints.addPoints(userData.dict)
UserItemPoints.addPoints(itemData.dict)

UserItemPoints.dict
# np.concatenate((A, B))


{1: {'vector': array([1.  , 2.  , 3.68]), 'type': 'user'},
 2: {'vector': array([1.  , 2.  , 3.68]), 'type': 'user'},
 3: {'vector': array([1.  , 2.  , 3.68]), 'type': 'user'},
 101010: {'vector': array([1, 2, 3]), 'type': 'item'},
 202020: {'vector': array([1, 2, 4]), 'type': 'item'}}

### 4. Predicciones del modelo $\hat{Y}(\lambda)$:

In [56]:
def recommend_n_items_to_user(user_id:int, n:int) -> list():
    global UserItemPoints
    userVector = userData.dict[user_id]["vector"]
    userPoint = User(user_id, userVector)
    return nearest_n_points(UserItemPoints, userPoint, n, _type="item")

def recommend_n_items_to_item(item_id:int, n:int) -> list():
    global UserItemPoints
    itemVector = itemData.dict[item_id]["vector"]
    itemPoint = Item(item_id, itemVector)
    return nearest_n_points(UserItemPoints, itemPoint, n, _type="item")

def nearest_n_points(context:Points, point:Point, n:int,
                      _type="item", _lambda=1, lambda_rate=1) -> list():

    # Devuelve los n puntos cercanos al punto dado
    
    # Elimino el punto elegido del contexto (si estuviera)
    if point.id in context.dict.keys():
        context.dict.pop(point.id)

    num_points = context.numPoints()

    # Caso invalido
    if num_points < n:
        raise Exception ("Context is shorter than the points you require")

    # Casos validos
    if n == 0:
        return []
    elif num_points == n:
        return context.getPointMatrix().tolist()    

    collected = points_in_centred_ksphere(context, point, _lambda)
    if len(collected) < n:        
        # Elimino los puntos extraidos del context
        for p in collected:
            context.dict.pop(p.id)
        
        # Aumento el tamaño del radio
        _lambda += lambda_rate

        return collected.extend(
            nearest_n_points(context, point, n-len(collected), _type, _lambda, lambda_rate)
        )

    return collected[:n]

def points_in_centred_ksphere(context:Points, center:Point, _lambda:float,
                               truncate=-1, _type="item") -> list():
    # Devuelve todos los puntos en la k-esfera de radio _lambda, centrada en el punto center
    # si se pasa un truncate, hay un stop cuando se colectan esa cantidad de puntos
    collect = list()
    for key in context.dict.keys():

        p_data = context.dict[key]
        ptype = p_data["type"]

        if ptype == _type:
            pvector = p_data["vector"]
            point = Point(key, pvector)        
            if (distance(point, center) <= _lambda):
                collect.append(point)

        if truncate > 0 and len(collect) >= truncate:
            return collect

    return collect

def distance(point_a:Point, point_b:Point) -> float:
    return np.linalg.norm(point_a.vector - point_b.vector)

### 5. Testing:

Idem al testing en sar model

In [63]:
# nearest_n_points(UserItemPoints, userData.dict[1], 3)
# recommend_n_items_to_user(1,2)

# UserItemPoints.shape()
# recommend_n_items_to_user(1,2)

points_in_centred_ksphere(UserItemPoints,itemA,_lambda=1000)

[<__main__.Point at 0x7f69f7e3a5c0>, <__main__.Point at 0x7f69f7e3aa40>]